# JETANK ColorRecognition

This document is used to obtain the HSV value of a certain pixels in the frame, which can be used for the subsequent color tracking function. 

The HSV color wheel sometimes appears as a cone or cylinder, but always with these three components: Hue is the color portion of the model, expressed as a number from 0 to 360 degrees(0 to 180 in OpenCV): Red falls between 0 and 60 degrees. Yellow falls between 61 and 120 degrees. Green falls between 121 and 180 degrees. Cyan falls between 181 and 240 degrees.

While an RGB color value is specified with: rgb (red, green, blue). Each parameter (red, green, and blue) defines the intensity of the color as an integer between 0 and 255. For example, rgb (0, 0, 255) is rendered as blue, because the blue parameter is set to its highest value (255) and the others are set to 0.

Therefore, in the color recognition function, in order to avoid the impact of environmental white balance and light brightness difference, we use the HSV color space to match colors.


# Import camera function libraries

After running the following code block, wait a while and wait for the camera to initialize. After the initialization is successful, a 300x300-sized real-time video screen will appear below the code block.

You can right-click on this screen and click `Create New View for Output`, so that you can place the camera screen in the window again. Even if you browse to other part of the document, you can still watch the camera screen at any time. This method applies to other widgets.

The initialization may fail due to running this code block multiple times. The solution is already included in `jetbot.Camera`, you only need to restart the Kernel, but be careful not to use the circle arrow above the tab, chances are the camera will still fail to initialize.

It is a recommended method to restart the Kernel:
In `File Browser` on the left, right-click on the `*.ipynb` file with a green dot in front (the green origin indicates that the Kernel is running), select `Shut Down Kernel`, and you will find a green dot disappears, then close this tab and double-click the `*.ipynb` file that was closed just now to restart the kernel.

Run the following code again, and the camera should be initialized normally.

In [1]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image_widget = ipywidgets.Image()  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

# Convert the frame to HSV color space

The following code is used to convert the frame into HSV color space, and draw some elements on the frame to facilitate the alignment of the pixel that needs to be sampled.

Note that the frame you see will not change after running the following code. Only after the value of `image_widget.value` is changed and the function `camera.observe()` is called can you observe the final effect. The former value is used to display the processed frame, and the latter function is used to call the image processing related methods immediately after the new frame is collected. 

In [2]:
import cv2

# Import the collected frames in this function and process them.
def colorSelect(imageInput):
    # Convert the imported frame to HSV color space.
    hsv = cv2.cvtColor(imageInput, cv2.COLOR_BGR2HSV)
    
    # Blur effect, to reduce the interference caused by the noise.
    hsv = cv2.blur(hsv, (15, 15))
    
    # Get the HSV parameters of the center pixel. 
    # Since the size of the screen is 300x300, the position of the intermediate pixel is (150, 150).
    hsvValue = hsv[150, 150]
    
    # Draw a 20x20 rectangle in the middle to aim at the object to be sampled.
    cv2.rectangle(imageInput,(140,140),(160,160),(255,255,255),1)
    
    # Displayed the HSV parameter on the frame.
    cv2.putText(imageInput,"HSV:{}".format(hsvValue),(80,180), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
    
    # Return the frame.
    return imageInput

# Process video frames and display

After running the following code, you can see that the color of the frame has changed, indicating that the video frame has been successfully processed by the `colorSelect()` function.

In [3]:
def execute(change):
    global image_widget
    image = change['new']
    image_widget.value = bgr8_to_jpeg(colorSelect(image))
    
execute({'new': camera.value})
camera.unobserve_all()
camera.observe(execute, names='value')

There will be a small white square in the video screen, use this small square to align the object you want to get the HSV value.
After aligning the object, the HSV value under the square will change within a certain range. Record the maximum and minimum values of these three values, for example:
colorUpper = [44, 255, 255]
ColorLower = [24, 100, 100]

## Question: Why do we need this value to change rapidly instead of sampling multiple times to average or take the average of multiple points?
Because after blurring, this pixel is actually the result of averaging with surrounding pixels. We need to consider the value jump range of individual pixels in this area, so we can’t collect multiple pixels and then average In this way, the HSV value range of the color in the actual screen will be greatly reduced, and the work efficiency of subsequent color recognition related functions will be affected.

# Turn off this function processing and stop the camera
Run the following code block to turn off the image processing function.

In [4]:
camera.unobserve(execute, names='value')

Again, let's close the camera conneciton properly so that we can use the camera in the later notebook.

In [4]:
camera.stop()